In [1]:
from collections import defaultdict
from math import ceil
import random
import sys
import os
import re

import cv2
import numpy as np
from tqdm import tqdm
from loguru import logger
from natsort import natsorted
import matplotlib.pyplot as plt

from keras.models import Model, load_model
from keras.callbacks import TensorBoard
from keras.models import model_from_json
from keras.layers import Input, Conv2D, Dense, MaxPooling2D, UpSampling2D

%matplotlib inline

ImportError: numpy.core.multiarray failed to import

ImportError: numpy.core.multiarray failed to import

In [2]:
dim = 320

def get_model(img_width = dim, img_heigth = dim):

    input_img = Input(shape=(img_width, img_heigth, 1))
    x = Conv2D(16, (3, 3), activation='relu', padding ='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding ='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding ='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding ='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    #6x6x32 -- bottleneck

    x = UpSampling2D((2, 2))(encoded)
    x = Conv2D(128, (3, 3), activation='relu', padding ='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding ='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding ='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding ='same')(x)
    decoded = Conv2D(1, (3, 3), activation='relu', padding ='same')(x)

    #Create model
    autoencoder = Model(input_img, decoded)

    return autoencoder, encoded


def load_dataset(path, img_width = dim, img_heigth = dim):
    
    folders = natsorted(os.listdir(path))

    X = []
    Y = []
    names = []

    for folder in folders:

        if os.path.isdir(path + folder):
            frames = natsorted(os.listdir(path + folder))
            logger.debug("Loading: " + folder)

            for frame in frames:
                # load a new frame
                new_frame = cv2.imread(path + folder + "/" + frame)

                # convert to grayscale
                new_frame = cv2.cvtColor(new_frame, cv2.COLOR_RGB2GRAY)
                # resize frame
                new_frame = cv2.resize(new_frame, (img_width, img_heigth))

                X.append(new_frame.astype("float16")/255.)
                Y.append(folder)
                names.append(frame.split('_')[0])
    X = np.array(X).reshape((-1, img_width, img_heigth, 1))

    return X, Y

# this method load all frame names within the action folders
def split_by_person(path, frames, labels, names, sorted_names):

    folders = natsorted(os.listdir(path))
    persons = []

    # regex for detection of the names belonging to each dataset 
    train_names = re.compile('|'.join(sorted_names[0][2]))
    test_names = re.compile('|'.join(sorted_names[1][2]))
    eval_names = re.compile('|'.join(sorted_names[2][2]))

    train_data = []
    train_labels = []
    
    test_data = []
    test_labels = []

    eval_data = []
    eval_labels = []

    for _, frame in zip(tqdm(range(len(frames))), frames):
        for label in labels:
            for name in names:
                if re.match(train_names, name):
                    train_data.append(frame)
                    train_labels.append(label)
                elif re.match(test_names, name):
                    test_data.append(frame)
                    test_labels.append(label)
                elif re.match(eval_names, name):
                    eval_data.append(frame)
                    eval_labels.append(label)


#     train_data = np.array(train_data)
#     test_data = np.array(test_data)
#     eval_data = np.array(eval_data)


    return train_data, train_labels, test_data, test_labels, eval_data, eval_labels

In [3]:
autoencoder, encoded = get_model()
autoencoder.load_weights('model.h5')
autoencoder.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 320, 320, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 320, 320, 16)      160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 160, 160, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 160, 160, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 80, 80, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 80, 80, 64)        18496     
_________________________________________________________________
max_

In [4]:
for _ in range(9):
    autoencoder.layers.pop()
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 320, 320, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 320, 320, 16)      160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 160, 160, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 160, 160, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 80, 80, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 80, 80, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 40, 40, 64)        0         
__________

In [5]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 320, 320, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 320, 320, 16)      160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 160, 160, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 160, 160, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 80, 80, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 80, 80, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 40, 40, 64)        0         
__________

In [6]:
path = './data/'

names = ["daria", "denis", "eli", "ido", "ira", "lena", "lyova", "moshe", "shahar"]

sorted_names = [('train_list', 0.7, ['ira', 'denis', 'moshe', 'daria', 'lyova', 'ido']),
                ('test_list', 0.2, ['lena', 'shahar']),
                ('eval_list', 0.1, ['eli'])]

frames, labels = load_dataset(path)

2019-07-05 18:49:43.689 | DEBUG    | __main__:load_dataset:45 - Loading: bend_subtracted
2019-07-05 18:49:45.965 | DEBUG    | __main__:load_dataset:45 - Loading: jack_subtracted
2019-07-05 18:49:47.443 | DEBUG    | __main__:load_dataset:45 - Loading: jump_subtracted
2019-07-05 18:49:48.345 | DEBUG    | __main__:load_dataset:45 - Loading: pjump_subtracted
2019-07-05 18:49:49.381 | DEBUG    | __main__:load_dataset:45 - Loading: run_subtracted
2019-07-05 18:49:50.264 | DEBUG    | __main__:load_dataset:45 - Loading: side_subtracted
2019-07-05 18:49:51.100 | DEBUG    | __main__:load_dataset:45 - Loading: skip_subtracted
2019-07-05 18:49:52.052 | DEBUG    | __main__:load_dataset:45 - Loading: walk_subtracted
2019-07-05 18:49:53.431 | DEBUG    | __main__:load_dataset:45 - Loading: wave1_subtracted
2019-07-05 18:49:54.726 | DEBUG    | __main__:load_dataset:45 - Loading: wave2_subtracted


In [7]:
# omitting the labels
train_data, _, test_data, _, eval_data, _ = split_by_person(path, frames, labels, names, sorted_names)

100%|██████████| 5594/5594 [06:39<00:00, 13.21it/s]


In [ ]:
predicted_frames = autoencoder.predict(eval_data)

In [ ]:
np.shape(predicted_frames)

In [ ]:
def compare_results(frames, predicted, n_frames = 5):

    columns = 2
    fig = plt.figure(figsize=(50,50))

    for i in range(0, 2 * n_frames ):
        for j in range(0, columns):        
            fig.add_subplot(2 * n_frames, columns, i + 1)

            if j % 2 == 0:
                print('#'*10)
                print('frame plotted, index:', i)
                plt.imshow(frames[i, : , : , 0], cmap = 'gray')
            else:
                print('%'*10)
                print('predicted frame plotted, index:', i)
                plt.imshow(predicted[i, : , : , 0], cmap = 'gray')
        
    plt.show()
    fig.savefig('resultado.png', dpi = fig.dpi)


compare_results(eval_data, predicted_frames, 5)